## 前回（20201222）のおさらい

### 21. データ解析を行う際のTips

#### <font color = blue>**1.** </font>EDA (探索的データ解析)

In [ ]:
## irisの例で実演
# データを読み込む
from sklearn.datasets import load_iris
iris = load_iris()

# DataFrame形式に変換
import pandas as pd
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

In [ ]:
'''
# データの先頭表示
iris_df.head(10)

# データの末尾表示
iris_df.tail()

# データの要約表示
iris_df.info()

# データの次元数（何行/何列）表示
print('There are {} rows and {} columns in data'.format(iris_df.shape[0], iris_df.shape[1]))

# データのカラム（列）名を取得
iris_df.columns

In [ ]:
'''
# 後々利用する可能性があるので、型タイプに合わせたカラム名のリストを保持しておくと便利かも
# includeを型タイプ（int, float64, etc...）にすることも可能

obj_columns = iris_df.select_dtypes(include=['object']).columns
print("obj_columns : ", obj_columns, "\n")

num_columns = iris_df.select_dtypes(include=['number']).columns
print("num_columns : \n", num_columns)

In [ ]:
'''
# 各列データの欠損値の個数をカウント
iris_df.isnull().sum()

# 各列データの型タイプ確認
iris_df.dtypes

In [ ]:
## データを図示して各特徴量の分布を確認
# 外れ値の有無/割合など

# 各列（特徴量）のヒストグラムの場合
import matplotlib.pyplot as plt
iris_df.hist(bins=10, figsize=(10,10))
plt.show()

In [ ]:
## 相関行列のヒートマップ
import seaborn as sns
sns.heatmap(iris_df.corr(),
            annot=True,
            cmap='YlGnBu')
plt.show()

##### pandas_profiling

In [ ]:
## なお、ここまでの作業をほぼ全て自動でまとめてくれる便利なライブラリが公開されている

In [ ]:
!pip show pandas_profiling

In [ ]:
!pip install git+https://github.com/pandas-profiling/pandas-profiling.git

In [ ]:
#!pip install pandas_profiling
from pandas_profiling import ProfileReport

#import warnings
#warnings.filterwarnings('ignore')

profile = ProfileReport(iris_df)
profile

#### <font color = blue>**2.** </font>Feature Engineering (特徴量エンジニアリング / 特徴選択）

In [ ]:
## 特徴量の重要度の推定
# どの特徴量（説明変数）が、target（目的変数）にとって重要であるかを、ランダムフォレスト(RandomForest)を用いて算出する

In [ ]:
## 外れ値処理
# 特徴量に外れ値が含まれている場合、一般的にモデルの精度に影響しやすいので、外れ値処理は必須
# データ自体を使わない（切り捨てる）、規格化/正規化/標準化する、など

In [ ]:
## target（目的変数）の分布を確認
# 正規分布にしたがっているかどうか
# 目的変数が正規分布に従っているか否かは、一般的に機械学習モデルに大きく影響するため重要
# 対数変換および差分変換を行ない、正規分布に従うように加工/整形する

#### <font color=red>**advanced task** : </font> 以上に挙げた作業の流れで　Feature Engineering　を行う

$\downarrow \downarrow$ 適当な化学的データが手元になければこちらでどうぞ $\downarrow \downarrow$\
https://raw.githubusercontent.com/jiai-tus/FirstTerm/main/20201222/datasets/Grisoni_et_al_2016_EnvInt88.csv

出展 : https://archive.ics.uci.edu/ml/datasets/QSAR+Bioconcentration+classes+dataset

### 22. k近傍法 (k-NearestNeighbor)
<font color = blue>**1.** </font>分類 (classification)

ライブラリ : sklearn.neighbors.KNeighborsClassifier

$\downarrow \downarrow$ 公式リファレンス $\downarrow \downarrow$\
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

#### <font color=green> **1.1.** </font> データの準備

In [ ]:
# irisデータをロード
from sklearn.datasets import load_iris
iris = load_iris()

# データフレーム形式に変換
# カラム名を説明変数に設定
import pandas as pd
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

#iris_df.head( )

In [ ]:
'''
# 目的変数を確認
iris.target

## 目的変数（＝アヤメの種類）は iris.target にセットされているが、数字の状態
# 人間が読める(=human-redable)種類名の表記としての目的変数は iris.target_names
iris.target_names

In [ ]:
# 目的変数部分だけをデータフレーム形式に変換
iris_target_data = pd.DataFrame(iris.target, columns=['Species'])

In [ ]:
'''
# データの確認
print(iris_df)
print(iris_target_data)

In [ ]:
'''
## 目的変数を数値 [0 1 2] ではなく種類名 ['setosa' 'versicolor' 'virginica'] としたい場合
iris_df2 = pd.DataFrame(iris.data, columns=iris.feature_names)

iris_df2['species'] = pd.Categorical.from_codes(iris.target, iris.target_names) 

print(iris_df2.head(10))

In [ ]:
## データの前処理（の一例）
## 特徴量を抽出/選別 (feature engineering)

# 説明変数ごとに、その変数の意味を踏まえて、必要に応じて Data Cleaning
# >> 目的変数に対する偏りを補正
# >> 非数値であれば何らかのルールに従い数値に変換
# >> 欠損値があれば何らかのルールに従い補充
# >> 規格化、標準化、正規化、など

# 説明変数の次元を圧縮
# >> PCAなど

## iris はそのまま使用して十分な精度が出るようにすでに整えられたデータセット

#### <font color=green> **1.2.** </font> データセットの分割（ホールドアウト）

In [ ]:
# ライブラリをインポート
from sklearn.model_selection import train_test_split

# ホールドアウト実行
X_train, X_test, Y_train, Y_test = train_test_split(iris_df, iris_target_data)

# デフォルトでは 学習用データ部分 ： テストデータ部分 = 3 : 1 に分かれる (test_size=0.25)

In [ ]:
'''
# データ型を確認
print("X_train : ", type(X_train), "\n",
      "X_test : ", type(X_test), "\n",
      "Y_train : ", type(Y_train), "\n",
      "Y_train : ", type(Y_train), "\n")

In [ ]:
'''
# データの中身を見てみる
print(X_train, "\n")
print(X_test, "\n")
print(Y_train, "\n")
print(Y_test)

In [ ]:
'''
# 分割された様子を図示する
X_df = X_train.copy()
X_df['Species'] = Y_train

import seaborn as sns
sns.pairplot(X_df, hue='Species', height=2)

X_df2 = X_test.copy()
X_df2['Species'] = Y_test

sns.pairplot(X_df2, hue='Species', height=2)

In [ ]:
## 目的変数をDataFrame形式からnumpy配列へ変換
# 目的変数が1次元の場合、scikit-learn は行ベクトル（1d array）を推奨してくるため

import numpy as np
Y2_train = np.array(Y_train.Species)

#print(Y2_train)

#### <font color=green> **1.3.** </font> k近傍法を実行

In [ ]:
# ライブラリのインポート
from sklearn.neighbors import KNeighborsClassifier

# 試しに k=6 でk近傍法をしてみよう
knn = KNeighborsClassifier(n_neighbors=6)

# 学習用データのみを与えて .fit 実行
knn.fit(X_train, Y2_train)

In [ ]:
# テスト用データによる予測結果を取得
Y_pred = knn.predict(X_test)

#Y_pred

In [ ]:
## どれくらいの精度で予測できているのかを確認しよう
# ライブラリのインポート
from sklearn import metrics
import pandas.testing as tm

# 正解（テスト用データの目的変数）に対する予測結果の正答率を計算して表示する
metrics.accuracy_score(Y_test, Y_pred)

#### <font color=green> **1.4.** </font> ハイパーパラメータ k について

In [ ]:
## k の値によって正答率がどう変わるかを調べよう

# 正答率を保存する入れ物を作成
accuracy_list = []

# k = 1〜100 でk近傍法を行い、それぞれの正答率を取得して保存（格納）
k_range = range(1, 101)

for k in k_range:
  knn = KNeighborsClassifier(n_neighbors=k)
  Y2_train = np.array(Y_train.Species)
  knn.fit(X_train, Y2_train)
  Y_pred = knn.predict(X_test)
  accuracy_list.append(metrics.accuracy_score(Y_test, Y_pred))

In [ ]:
# わかりやすくグラフで図示する
import matplotlib.pyplot as plt

# 背景や罫線、目盛りスケールを自動でいい感じにしてくれる
sns.set()

plt.figure(figsize=(8, 6))
plt.plot(k_range, accuracy_list)
plt.xlabel('k-nn')
plt.ylabel('accuracy')
plt.show()

#### <font color=green> **1.5.** </font> 交差検証（Cross-validation）とグリッドサーチ(Grid Search) 
https://scikit-learn.org/stable/modules/cross_validation.html

In [ ]:
# 学習用データ/テスト用データ の分割を網羅的に行う
# 上記の分割のそれぞれにおいて、学習用データをさらに訓練データ/検証データへ網羅的に分割し、比較/統合することで予測精度を高める

## k近傍法の場合、ハイパーパラメータである k に大きく依存するので、もっとも精度が高くなるようにしたい

In [ ]:
## 目的変数を全種類満遍なく含みつつ、ランダムシャッフルされる分割で行ってみる
# ライブラリのインポート
from sklearn.model_selection import StratifiedShuffleSplit

## 学習用データ/テスト用データ の分割
# 学習用 : テスト用 = 120 : 30 で５通り
sss1 = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

## 学習用データをさらに訓練データ/検証データへ網羅的に分割
# 訓練用 : 検証用 = 100 : 20 で6通り
sss2 = StratifiedShuffleSplit(n_splits=6, test_size=20, random_state=1)

In [ ]:
## 上記の 5通り × 6通り = 30通り 全てで最適な k の値を検証

i = 1
plt.figure(figsize=(24, 30))

for train_index1, test_index1 in sss1.split(iris_df, iris_target_data):
  #print("TRAIN:", train_index1, "\n TEST:", test_index1, "\n")
  X_learn, X_test = iris_df.loc[train_index1], iris_df.loc[test_index1]
  y_learn, y_test = iris_target_data.loc[train_index1], iris_target_data.loc[test_index1]

  for train_index2, test_index2 in sss2.split(X_learn, y_learn):
    X_train, X_verify = iris_df.loc[train_index2], iris_df.loc[test_index2]
    y_train, y_verify = iris_target_data.loc[train_index2], iris_target_data.loc[test_index2]

    # 正答率を保存する入れ物
    accuracy_list = []

    # k = 1〜20 でk近傍法を行い正答率を取得
    k_range = range(1, 21)

    for k in k_range:
      knn = KNeighborsClassifier(n_neighbors=k)
      y2_train = np.array(y_train.Species)
      knn.fit(X_train, y2_train)
      y_pred = knn.predict(X_verify)
      # 網羅的に分割した1つの場合における、訓練用データを学習に使用し検証用データで正答率を算出
      accuracy_list.append(metrics.accuracy_score(y_verify, y_pred))

    # 背景や罫線、目盛りスケールを自動でいい感じにしてくれる
    sns.set()

    plt.subplot(5,6,i)
    plt.plot(k_range, accuracy_list)
    #plt.title(("TRAIN : ", train_index, " TEST : ", test_index))
    plt.xlabel('k-nn')
    plt.ylabel('accuracy')
    i += 1

plt.show()

In [ ]:
## k=3,4,5 が候補か。。。

In [ ]:
# 学習用/テスト用に網羅的に分割した各場合において、学習用データで学習しテスト用データで正答率を算出

i = 1
plt.figure(figsize=(25, 6))

for train_index, test_index in sss1.split(iris_df, iris_target_data):
  #print("TRAIN:", train_index1, "\n TEST:", test_index1, "\n")
  X_learn, X_test = iris_df.loc[train_index], iris_df.loc[test_index]
  y_learn, y_test = iris_target_data.loc[train_index], iris_target_data.loc[test_index]

  # 正答率を保存する入れ物
  accuracy_list = []

  # k = 2〜6 でk近傍法を行い正答率を取得
  k_range = range(2, 7)

  for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    y2_learn = np.array(y_learn.Species)
    knn.fit(X_learn, y2_learn)
    y_pred = knn.predict(X_test)
    accuracy_list.append(metrics.accuracy_score(y_test, y_pred))

  # 背景や罫線、目盛りスケールを自動でいい感じにしてくれる
  sns.set()

  plt.subplot(1,5,i)
  plt.plot(k_range, accuracy_list)
  #plt.title(("TRAIN : ", train_index, " TEST : ", test_index))
  plt.xlabel('k-nn')
  #plt.ylabel('accuracy')
  i += 1

plt.show()

In [ ]:
## k = 3 or 5 とするのがもっとも予測精度が高い
# しかしあくまで手元にある150個のデータに対して、の話
# 全く未知の iris のデータに対する予測精度（汎化性能）は、やってみないと誰にもわからない

In [ ]:


####
## 用意されているライブラリにお任せするなら...
####



In [ ]:
## 交差検証のスコアのみ知りたい場合
from sklearn.model_selection import cross_val_score

scores_k3 = cross_val_score(KNeighborsClassifier(n_neighbors=3),  # 学習モデルを指定
                            iris.data,  # 説明変数のデータ
                            iris.target,  # 目的変数のデータ
                            cv = 5  # ホールドアウトの分割数。default=5
                            )

scores_k3

In [ ]:
scores_k5 = cross_val_score(KNeighborsClassifier(n_neighbors=5),  # 学習モデルを指定
                            iris.data,  # 説明変数のデータ
                            iris.target,  # 目的変数のデータ
                            cv = 5  # ホールドアウトの分割数。default=5
                            )

scores_k5

In [ ]:
## 交差検証のスコアだけでなく、学習結果（予測モデル）を利用したい場合
from sklearn.model_selection import cross_validate

cv_results = cross_validate(KNeighborsClassifier(n_neighbors=3),  # 学習モデルを指定
                            iris.data,  # 説明変数のデータ
                            iris.target,  # 目的変数のデータ
                            cv = 5, # ホールドアウトの分割数。default=5
                            return_train_score = True,  # 学習データのスコアを返すかどうか。default=False
                            return_estimator = True # 予測モデルを返すかどうか。default=False
                            )

In [ ]:
'''
# データを取り出すkey値一覧を表示
sorted(cv_results.keys())

# 計算に要した時間。普通は不要
print("fit_time \t\t: ", cv_results['fit_time'])
print("score_time \t: ", cv_results['score_time'])

# テストデータに対する正答率
cv_results['test_score']

# 学習データに対する正答率
cv_results['train_score']

## ホールドアウトの分割ごとに作成された各モデルオブジェクトが、tupleで格納されている
cv_results['estimator']

In [ ]:
## グリッドサーチ
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(KNeighborsClassifier(),  # 学習モデルのメソッドを指定
                   cv = 5,  # ホールドアウトの分割数。default=5
                   param_grid={"n_neighbors" : range(1,21)} # 指定したメソッドの引数名とグリッドサーチしたい範囲をdictionary形式で指定
                   )

# 教師データを渡して学習実行
clf.fit(iris.data, iris.target)                   

In [ ]:
'''
# 結果を取り出すkey値一覧を表示
sorted(clf.cv_results_.keys())

# 結果一覧
clf.cv_results_

In [ ]:
# 最適な予測モデル
clf.best_estimator_

In [ ]:
# 最適な予測モデルの正答率
clf.best_score_

#### <font color=green> **1.6.** </font> 混同行列（Confusion Matrix)

In [ ]:
## k = 3 and 5 の時に混同行列を求めてみる

# 改めてホールドアウト
X_train, X_test, Y_train, Y_test = train_test_split(iris_df, iris_target_data)

# 目的変数を行ベクトル（1d array）化
Y2_train = np.array(Y_train.Species)

In [ ]:
# k = 3 のk近傍法を実行し予測結果取得
knn_3 = KNeighborsClassifier(n_neighbors=3)
knn_3.fit(X_train, Y2_train)
Y_pred_3 = knn_3.predict(X_test)

In [ ]:
# k = 5 のk近傍法を実行し予測結果を取得 
knn_5 = KNeighborsClassifier(n_neighbors=5)
knn_5.fit(X_train, Y2_train)
Y_pred_5 = knn_5.predict(X_test)

In [ ]:
# 混同行列を計算してくれるライブラリをインポート
from sklearn.metrics import confusion_matrix

# k = 3 の混同行列を算出
cm_3 = confusion_matrix(Y_test, Y_pred_3)

In [ ]:
# k = 3 の混同行列を表示 or 図示
cm_3

In [ ]:
sns.heatmap(cm_3, annot=True)

In [ ]:
# k = 5 の混同行列を算出
cm_5 = confusion_matrix(Y_test, Y_pred_5)

In [ ]:
# k = 5の混同行列を表示 or 図示
cm_5

In [ ]:
sns.heatmap(cm_5, annot=True)

In [ ]:
# なお、混同行列の計算結果を図示する専用ライブラリがある（seaborn.heatmapと同じ感じ）
from sklearn.metrics import ConfusionMatrixDisplay


# k = 3 の混同行列を表示

# 念のため各種パラメータをデフォルトに
sns.set_style("ticks")

print("k = 3")
cm_display = ConfusionMatrixDisplay(cm_3, display_labels=iris.target_names).plot()

In [ ]:
# k = 5の混同行列を表示

# 念のため各種パラメータをデフォルトに
sns.set_style("ticks")

print("k = 5")
cm_display = ConfusionMatrixDisplay(cm_5, display_labels=iris.target_names).plot()

## 本日の内容（続きから）


### 22. k近傍法 (k-NearestNeighbor)
<font color = blue>**2.** </font>回帰 (regression)

ライブラリ : sklearn.neighbors.KNeighborsRegressor

$\downarrow \downarrow$ 公式リファレンス $\downarrow \downarrow$\
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html

##### <font color=green> **2.1.** </font> データの準備

In [ ]:
# bostonデータをロード
from sklearn.datasets import load_boston
boston = load_boston()

In [ ]:
# データフレーム形式に変換
# カラム名を説明変数に設定
import pandas as pd
boston_df = pd.DataFrame(boston.data, columns = boston.feature_names)

print(boston_df.head())

In [ ]:
# 目的変数を確認
print(boston.target)

In [ ]:
# 説明を表示
print(boston.DESCR)

In [ ]:
# 目的変数部分だけをデータフレーム形式に変換
boston_target_data = pd.DataFrame(boston.target, columns=['MEDV'])

In [ ]:
# 説明変数 vs 目的変数 の散布図を表示
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(25, 15))
sns.set()
plt.rcParams['lines.markersize'] = 2

i = 1
for columns in boston.feature_names:
  plt.subplot(3,5,i)
  plt.scatter(boston_df[columns], boston_target_data)
  plt.xlabel("{}".format(columns))
  i += 1

plt.show()

##### <font color=red> task : </font> boston のデータの前処理を行う

正解はないので自由な発想で様々試みてみてほしい

##### <font color=green> **2.2.** </font> データセットの分割（ホールドアウト）

In [ ]:
# ライブラリのインポート
from sklearn.model_selection import train_test_split

In [ ]:
# ホールドアウト実行
X_train, X_test, Y_train, Y_test = train_test_split(boston_df, boston.target)

# デフォルトでは 学習用データ部分 ： テストデータ部分 = 3 : 1 に分かれる (test_size=0.25)

##### <font color=red> task : </font> 分割された様子を図示する

##### <font color=green> **2.3.** </font> k近傍法を実行

In [ ]:
## 今回はコードの紹介がメインなので、前処理などはせずそのままのデータを使用

In [ ]:
# ライブラリのインポート




In [ ]:
# デフォルト値（k=5）で回帰するインスタンス生成




In [ ]:
# 学習用データのみを与えて .fit 実行




In [ ]:
# テスト用データによる予測結果を取得




In [ ]:
## 予測結果と正解（テスト用データの目的変数）との誤差を計算する

In [ ]:
## 平均二乗誤差
# ライブラリのインポート




In [ ]:
# 計算実行




In [ ]:
# 計算結果を表示
print('平均二乗誤差\t: ', mse)
print('その平方根\t: ', mse**(1/2))

In [ ]:
## 平均絶対誤差
# ライブラリのインポート




In [ ]:
# 計算実行




In [ ]:
# 計算結果を表示
print('平均絶対誤差\t: ', mae)

In [ ]:
## 予測結果を縦軸、正解（テスト用データの目的変数）を横軸として散布図を表示
# 予測と正解が完全に一致していれば、 y = x の直線上に分布するので、その線も引く

plt.figure(figsize=(8, 8))
sns.set()
plt.rcParams['lines.markersize'] = 5

plt.scatter(      )
plt.plot(       , color="red")
plt.xlabel(   )
plt.ylabel(   )

plt.show()

##### <font color=green> **2.4.** </font> ハイパーパラメータ k について

In [ ]:
## kの値による誤差の変化を調べる

# 平均絶対誤差を保存する入れ物を作成
mae_list = []

# 平均二乗誤差を保存する入れ物を作成
mse_list = []

In [ ]:
# k=1〜100 でk近傍法を行い、それぞれの誤差２種を取得して保存（格納）
k_range = range(1, 101)
for k in k_range:
  knn = KNeighborsRegressor(n_neighbors=k)
  knn.fit(X_train, Y_train)
  Y_pred = knn.predict(X_test)
  mae_list.append(mean_absolute_error(Y_test, Y_pred))
  mse_list.append(mean_squared_error(Y_test, Y_pred))

In [ ]:
# 並べて図示
plt.figure(figsize=(20, 8))
sns.set()

# 左側に平均絶対誤差:mae
plt.subplot(1,2,1)
plt.plot(k_range, mae_list)
plt.xlabel('k-nn')
plt.ylabel("mae")

# 右側に平均二乗誤差:mse
plt.subplot(1,2,2)
plt.plot(k_range, mse_list)
plt.xlabel('k-nn')
plt.ylabel("mse")

plt.show()

In [ ]:
# k=1〜20 を拡大表示（抽出）

# 並べて図示
plt.figure(figsize=(16, 6))
sns.set()

# 左側に平均絶対誤差:mae
plt.subplot(1,2,1)
plt.plot(k_range[0:20], mae_list[0:20], "bo-")
plt.xlabel('k-nn')
plt.ylabel("mae")

# 右側に平均二乗誤差:mse
plt.subplot(1,2,2)
plt.plot(k_range[0:20], mse_list[0:20], "bo-")
plt.xlabel('k-nn')
plt.ylabel("mse")

plt.show()

In [ ]:
## k=3,4 が候補か。。。

#### <font color=red> task : </font> boston のデータで交差検証/グリッドサーチを行い、最適な k を決定する

実行した前処理ごとにさらに比較できているとなお良い

## 23. 決定木 (DecisionTrees) と ランダムフォレスト (RandomForest)

### <font color = blue>**1.** </font>分類木
ライブラリ : sklearn.tree.DecisionTreeClassifier

$\downarrow \downarrow$ 公式リファレンス $\downarrow \downarrow$\
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

##### <font color=green> **1.1.** </font> データの準備

In [ ]:
# criterion : 分割基準。 gini or entropy を選択。(デフォルトでジニ係数)
# max_depth : 木の深さ。木が深くなるほど過学習し易いので、適当なしきい値を設定してあげる
# max_features ： 最適な分割をする際の特徴量の数
# min_samples_split ： 分岐する際のサンプル数
# random_state ： ランダムseedの設定。seedを設定しないと、毎回モデル結果が変わる

In [ ]:
# irisデータをロード
from sklearn.datasets import load_iris
iris = load_iris()

In [ ]:
# データフレーム形式に変換
# カラム名を説明変数に設定
import pandas as pd
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

In [ ]:
## ちなみにこのように書くと目的変数を一発で合体できる
iris_df["species"] = pd.Categorical.from_codes(iris.target, iris.target_names)

In [ ]:
iris_df.head()

In [ ]:
## 以下、1つのデータフレームに説明変数と目的変数がまとめられている時のコード例を想定

##### <font color=green> **1.2.** </font> データセットの分割（ホールドアウト）

In [ ]:
# ライブラリをインポート
from sklearn.model_selection import train_test_split

In [ ]:
# 説明変数を切り出す
features = iris_df.loc[:, iris.feature_names]

In [ ]:
# 目的変数を切り出す
label = iris_df["species"]

In [ ]:
# ホールドアウト実行
df_train, df_test, label_train, label_test = train_test_split(features, label)

# デフォルトでは 学習用データ部分 ： テストデータ部分 = 3 : 1 に分かれる (test_size=0.25)

##### <font color=green> **1.3.** </font> 決定木を実行

In [ ]:
# ライブラリのインポート




In [ ]:
##  モデルのインスタンス生成
# 引数はあえて適当に設定してみる
dtc = DecisionTreeClassifier(max_leaf_nodes=  ,
                            max_depth=  ,
                            min_samples_split=  ,
                            min_samples_leaf= ,
                            random_state= ,
                            )

In [ ]:
# 学習用データのみを与えて .fit 実行
dtc.fit(      )

In [ ]:
# テスト用データによる予測結果を表示
print("予測精度\t : ", dtc.score(df_test, label_test))

##### <font color=green> **1.4.** </font> モデルを可視化

In [ ]:
## 決定木モデルをDOTデータという規格に変換する
# ライブラリのインポート




In [ ]:
# 変換実行
dot_data = export_graphviz(   ,
                           filled=True,
                           rounded=True,
                           feature_names= ,
                           class_names= ,
                           special_characters=True)

In [ ]:
## DOTデータを描画するライブラリをインポート



In [ ]:
# ダイアグラムを描画
graph = pdp.graph_from_dot_data(dot_data)

In [ ]:
'''
画像ファイルとして出力（保存）したい場合

file_name = "/content/tree_visualization.png"
graph.write_pdf(file_name)

'''

In [ ]:
## ダイアグラムを表示するためのライブラリをインポート



In [ ]:
# 画面（コンソール）にダイアグラムを表示
Image(graph.create_png())

### <font color = blue>**2.** </font>回帰木
ライブラリ : sklearn.tree.DecisionTreeRegressor

$\downarrow \downarrow$ 公式リファレンス $\downarrow \downarrow$\
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

##### <font color=green> **2.1.** </font> データの準備

In [ ]:
# bostonデータをロード
from sklearn.datasets import load_boston
boston = load_boston()

In [ ]:
# データフレーム形式に変換
# カラム名を説明変数に設定
import pandas as pd
boston_df = pd.DataFrame(boston.data, columns = boston.feature_names)

In [ ]:
# 目的変数を追加
boston_df['MEDV'] = boston.target

In [ ]:
# 先頭５行を表示
print(boston_df.head()) 

In [ ]:
# データの形状を確認
print(boston_df.shape) 

##### <font color=green> **2.2.** </font> データセットの分割（ホールドアウト）

In [ ]:
# データセットをNumpy配列に変換




In [ ]:
# 説明変数と目的変数に分ける




In [ ]:
# ライブラリのインポート
from sklearn.model_selection import train_test_split

In [ ]:
# 今回は70%のデータを学習用データ、残りの30%のデータをテストデータとします
X_train, X_test, Y_train, Y_test = 

##### <font color=green> **2.3.** </font> 決定木を実行

In [ ]:
# ライブラリのインポート




In [ ]:
##  モデルのインスタンス生成
# 引数はあえて適当に設定してみる
dtr = DecisionTreeRegressor(max_leaf_nodes= ,
                            max_depth=  ,
                            min_samples_split=  ,
                            min_samples_leaf= ,
                            random_state= ,
                            )

In [ ]:
# 学習実行、結果を返り値で受け取る場合
model = dtr.fit(      )

##### <font color=green> **2.4.** </font> モデル評価

In [ ]:
## 学習したモデルによる予測の妥当性を評価したい
# 全サンプル（506個）の中からランダムに1個を抽出し、その特徴量から予測される価格と実際に観測された価格を比較する

In [ ]:
# 乱数生成
import random
random.seed(1)

In [ ]:
# ランダムにidを選定
id = random.randrange(0, X.shape[0], 1)
print(id)

In [ ]:
# 元のデータセットから該当サンプルを抽出
x = 

In [ ]:
# 行ベクトル（1d array）に変換
x = x.reshape(1,13)

In [ ]:
# 説明変数から住宅価格を予測
Y_pred = model.predict(   )

In [ ]:
# 実際のデータの住宅価格と比較
print("実際の価格\t : {}".format(boston_df.at[id, "MEDV"]),
      "\n予測価格\t\t : {}".format(float(Y_pred)))

In [ ]:
## モデル自体の汎用性 : モデルの予測値がどのくらい観測値の情報量を説明できているか
# 汎用性の指標として決定係数を確認する
# 決定係数R2 : 回帰分析において、観測値に対する予測値の説明力を表す指標。寄与率ともいう
# 0 から 1 までの値をとり、R2 が 1 に近いほどモデルが有効であることを意味します

In [ ]:
# 決定係数を算出する関数をインポート




In [ ]:
# テスト用の説明変数(X_test)をモデルに渡して予測値を算出
Y_pred2 = model.predict(    )

In [ ]:
# これらの予測値とテスト用の目的変数(Y_test)を渡して決定係数を算出
r2 = r2_score(      )
print("R2 score = ", r2)

##### <font color=green> **2.5.** </font> モデルを可視化

In [ ]:
## 分類木の可視化と全く同じです
# ライブラリをインポート





In [ ]:
# モデルをDOTデータに変換





In [ ]:
# ダイアグラムを描画





In [ ]:
# ダイアグラムを表示




### <font color = blue>**3.** </font>ランダムフォレスト
分類 $\rightarrow$ sklearn.ensemble.RandomForestClassifier

公式リファレンス $\rightarrow$ https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

\
回帰 $\rightarrow$ sklearn.ensemble.RandomForestRegressor

公式リファレンス $\rightarrow$ https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html


#### <font color = green>**3.1.** </font> ランダムフォレスト(Random Forests)のポイント

##### ★ランダムフォレストの特徴

- ランダムフォレストは簡単に言うと沢山の決定木を作成して、その多数決/平均をとるアルゴリズム
 - 過学習を抑える効果がある
 - 何千もの入力変数を削除せずにそのまま扱うことができる
 - 各変数の事前のスケーリングが不要
 - 交差検証や別個のテストデータでの検証をせずとも、アルゴリズム内で未知データに対する精度を推定することができる
 - 欠落値を推定する方法を持っているため、欠損値の多いデータでも精度を維持することができる
 - データ間の関係について情報を得ることができる

##### ★ポイント① ブートストラップ法 (Bootstrap Sampling)

https://ja.wikipedia.org/wiki/ブートストラップ法

- ランダムフォレストは決定木を沢山作って集合体である森にしている
 - その1つ1つの決定木を作成する際の訓練データの選び方がポイント
 - 全データの中から重複を許してサンプリングを行い（ブートストラッピング）、そのデータを決定木を作成する際の訓練データとする
 - 重複を許してサンプリングを行なうので、訓練データの中に同一データが含まれても構わない

##### ★ポイント② ノード（分岐点）作成する際に使用する変数を絞る
- データ全量の中に ***p*** 個の変数がある時、その全てを各ノード作成に使うのではなく、***m*** 個の変数をランダムに選んで使用する
 - 個数としては $\ m = \sqrt{p}\ $ 程度の数がよく用いられる

- この工程を繰り返しながら大量（通常は100本以上）の決定木を作成
- 作成された決定木の多様さが、ランダムフォレストをより効果的にしてくれる

##### ★ポイント③ バギング (Bagging : bootstrap aggregating)
- 1本1本の決定木による予測結果をアンサンブル（分類なら多数決、回帰なら平均）して最終的な出力とする
- 分散を減らし、過剰適合を避ける効果があるため、どんな手法にも使うことができるモデル平均化手法の一種

##### ★ポイント④ OOB (out-of-bag) 検証
- ブートストラップ法で選ばれなかったデータのことをOOB (out-of-bag) データと呼ぶ
 - 元データ数の大体 $\dfrac{1}{3}$ 程度の量になる
- このOOBデータによって、未知データに対する精度（汎化性能）を交差検証や別個のテストデータを要さず推定することができる
 - 作成したランダムフォレストにOOBデータを分類した際の分類誤差（OOBエラー : out-of-bag-error）
 - 回帰の場合は最小二乗誤差 (MSE) 等の指標
- この誤差指標に応じて使用する変数の数を調整することで精度の最大化を目指す

#### <font color = green>**3.2.** </font> iris の分類

In [ ]:
# 必要ライブラリのインポート
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# データをロード
iris = load_iris()

In [ ]:
# 説明変数
data = iris['data']

In [ ]:
# 目的変数
target = iris['target']

In [ ]:
# ホールドアウト
X_train, X_test ,Y_train, Y_test = 

In [ ]:
# 学習モデルのインスタンス生成
rfc = RandomForestClassifier(n_estimators=    , # 使用する決定木の数
                            oob_score =True)

In [ ]:
# 学習実行
rfc.fit(      )

In [ ]:
# 正答率を表示
print('test_data_accuracy \t : ', rfc.score(X_test, Y_test))
print('oob_data_accuracy \t : ', rfc.score(X_train, rfc.oob_decision_function_.argmax(axis = 1)))

#### <font color = green>**3.3.** </font> boston の回帰

In [ ]:
# 必要ライブラリのインポート
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# bostonデータをロード
boston = load_boston()

In [ ]:
# 説明変数
data2 = boston['data']

In [ ]:
# 目的変数
target2 = boston['target']

In [ ]:
# ホールドアウト
X_train, X_test ,Y_train, Y_test = 

In [ ]:
# 学習モデルのインスタンス生成
rfr = RandomForestRegressor(n_estimators=     , # 使用する決定木の数
                            oob_score =True)

In [ ]:
# 学習実行
rfr.fit(      )

In [ ]:
# R2 score を表示
print('test_data_R2 \t: ', rfr.score(X_test, Y_test))
print('oob_data_R2 \t: ', rfr.score(X_train, rfr.oob_prediction_))